PREGUNTA 1.

Primero, leeré los datos para ver su estructura y determinar qué columnas se utilizarán como características (X) y cuál será la variable objetivo (y).

In [1]:
%pip install pandas

import pandas as pd

dataset = pd.read_csv("./Spotify_final_dataset.csv")
cabezeras = dataset.head()
print(cabezeras)

print(dataset.info())


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
   Position    Artist Name                                   Song Name  Days  \
0         1   Post Malone   Sunflower  SpiderMan: Into the SpiderVerse  1506   
1         2    Juice WRLD                                 Lucid Dreams  1673   
2         3  Lil Uzi Vert                                XO TOUR Llif3  1853   
3         4       J. Cole                               No Role Modelz  2547   
4         5   Post Malone                                     rockstar  1223   

   Top 10 (xTimes)  Peak Position Peak Position (xTimes)  Peak Streams  \
0            302.0              1                  (x29)       2118242   
1            178.0              1                  (x20)       2127668   
2            212.0              1                   (x4)       1660502   
3              6.0              7                      0        659366   
4    

Para este tipo de dataset, se puede utilizar la herramienta de regresion logistica, con la que se puede utilizar para clasificar los datos en si han estado o no en el top 10, basandose en las caracteristicas del dato, como lo puede ser peak position. Por lo que tendremos solamente dos valores a predecir, esta herramienta será ideal.

Para poder lograr escalar los datos de manera eficiente, se va a utilizar un preprocesamiento de los datos con StandarScaler, el cual nos va a permitir caracterizar los datos a una escala con media 0 y desviación estandar 1. Esto va a facilitar la convergencia de los datos.

Luego para realizar el entrenamiento de la herramienta Regresion logistica, definimos que el 70% de los datos serán aplicados para el entrenamiento y el 30% para probar el rendimiento de este. De esta forma, se utilizarán datos ditintos a los utilizados para el entrenamiento de este. Además, como se puede ver en la configuracion de la herramienta, se le van a dar dos parametros, donde max_iter va a corresponder al aumento de iteraciones para asegurar que tenga tiempo de converger, y solver = 'lbfgs' donde este parametro va a optimizar la variables con un metodo llamado Limited-memory Broyden–Fletcher–Goldfarb–Shanno (L-BFGS), el cual va a mejorar la eficiencia, usando la matriz Hessiana para encontrar el mínimo de una función, pero solo una pequeña informacion (ya que limita la memoria). Luego de realizar estas configuraciones previas, se ejecuta el entrenamiento con la funcion fit(), dandole los parametros de entrenamiento.

Luego de esto ya se procede a evaluar el rendimiento de la herramienta ya entrenada dandole los datos de prueba.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

file_path = "./Spotify_final_dataset.csv"
dataset = pd.read_csv(file_path)

print(dataset.columns)

dataCleaned = dataset.drop(columns=['Position', 'Artist Name', 'Song Name'])

label_encoders = {}
for col in dataCleaned.select_dtypes(include=['object']).columns:
    labelEncoder = LabelEncoder()
    dataCleaned[col] = labelEncoder.fit_transform(dataCleaned[col])
    label_encoders[col] = labelEncoder

X = dataCleaned.drop(columns=['Top 10 (xTimes)'])
y = dataCleaned['Top 10 (xTimes)']

scalarDatos = StandardScaler()
X_escalados = scalarDatos.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_escalados, y, test_size=0.3, random_state=42)


model = LogisticRegression(max_iter=10000, solver='lbfgs', class_weight='balanced') 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.2f}")


conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)

Index(['Position', 'Artist Name', 'Song Name', 'Days', 'Top 10 (xTimes)',
       'Peak Position', 'Peak Position (xTimes)', 'Peak Streams',
       'Total Streams'],
      dtype='object')
Accuracy: 0.81
F1-Score: 0.84
Matriz de Confusión:
[[2615  197   19 ...    0    0    0]
 [   0   58    7 ...    0    0    0]
 [   0   23    4 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    1    0    0]
 [   0    0    0 ...    0    0    0]]


Como se puede observar, se calculan las metricas accuracy y f1 score, donde accuracy corresponde a la proporción de predicciones correctas respecto al total de observaciones, y f1 score corresponde a el cálculo de la media aritmética ponderada de la precisión y la sensibilidad.

Al ser un dataset con tantos datos, se logra optimizar con éxito el dataset, logrando hacer más eficiente esta herramienta. Como se ve, se obtiene un f1-score de 0.84, lo que indica que la presición del modelo  es muy eficiente. Además se logra obener un Accuracy de 0.81.

PREGUNTA 2.

A continuación se aplica el algoritmo de KNN, el cual consiste en aplicar el valor de K que va a definir el numero de vecinos que se concideran para realizar la clasificación de los datos. A continuación se realiza el algoritmo KNeighborsClassifier en python. Se tiene en cuenta el mismo procesamiento realizado anteriormente al dataset para optimizar y mantener la eficiencia del código.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

path = './Spotify_final_dataset.csv'
dataset = pd.read_csv(path)

dataCleaned = dataset.drop(columns = ['Position', 'Artist Name', 'Song Name'])

label_encoders = {}

label_encoders = {}
for col in dataCleaned.select_dtypes(include=['object']).columns:
    labelEncoder = LabelEncoder()
    dataCleaned[col] = labelEncoder.fit_transform(dataCleaned[col])
    label_encoders[col] = labelEncoder

X = dataCleaned.drop(columns=['Top 10 (xTimes)'])  
y = dataCleaned['Top 10 (xTimes)']

scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_escalado, y, test_size = 0.3, random_state = 42)

k_value = [3, 5, 7, 9, 11, 15, 18, 20, 50]

for k in k_value:
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'weighted')

    print(f"K: {k} - Accuracy: {accuracy:.2f} - F1-Score: {f1:.2f}")



K: 3 - Accuracy: 0.90 - F1-Score: 0.89
K: 5 - Accuracy: 0.90 - F1-Score: 0.89
K: 7 - Accuracy: 0.90 - F1-Score: 0.89
K: 9 - Accuracy: 0.90 - F1-Score: 0.88
K: 11 - Accuracy: 0.90 - F1-Score: 0.88
K: 15 - Accuracy: 0.90 - F1-Score: 0.88
K: 18 - Accuracy: 0.90 - F1-Score: 0.88
K: 20 - Accuracy: 0.90 - F1-Score: 0.88
K: 50 - Accuracy: 0.89 - F1-Score: 0.86


Como se puede observar, los resultados son bastante similares con respecto a la regresion logistica. Sin embargo, KNN posee una precisión mejor que en regresión logistica. Por lo que para este tipo de problema, o para la clasificación de estos datos, es mas adecuado aplicar KNN.

PREGUNTA 3

A continuación se procede a utilizar la herramienta DBSCAN y GMM. Donde se realizan algunos procesamiento en los datos para la optimización de las herramientas, ya que se esta trabajando con grandes volúmenes de datos. Esto se utiliza para poder realizar una comparación entre ambos algoritmos. Se van a utilizar las mismas etiquetas definidas en las preguntas 1 y 2, por lo que se podra notar diferencias en el funcionamiento de los algoritmos de clasificación.

DBSCAN es un algoritmo de clustering, donde se va a definir un eps, que va a corresponder a un radio de la vecindad que se debe conciderar para generar el cluster. Luego GMM es un modelo probabilistico que toma los datos como una mezcla de distribuciones gaussianas. Este va a tomar un promedio y una desviación estandar. 

In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

file_path = './Spotify_final_dataset.csv'
dataset = pd.read_csv(file_path)
dataCleaned = dataset.drop(columns=['Position', 'Artist Name', 'Song Name'])

label_encoders = {}
for col in dataCleaned.select_dtypes(include=['object']).columns:
    labelEncoder = LabelEncoder()
    dataCleaned[col] = labelEncoder.fit_transform(dataCleaned[col])
    label_encoders[col] = labelEncoder

# Separar características
X = dataCleaned.drop(columns=['Top 10 (xTimes)'])  # Ajustar según la columna objetivo de tu CSV
X_escalado = StandardScaler().fit_transform(X)

dbscan_conf = [
    {'eps': 0.3, 'min_samples': 5},
    {'eps': 0.5, 'min_samples': 5},
    {'eps': 0.3, 'min_samples': 10},
    {'eps': 0.5, 'min_samples': 10}
]


dbscan_results = {}
for conf in dbscan_conf:
    dbscan = DBSCAN(eps=conf['eps'], min_samples=conf['min_samples'])
    labels = dbscan.fit_predict(X_escalado)
    if len(set(labels)) > 1:
        silhouette = silhouette_score(X_escalado[labels != -1], labels[labels != -1])
    else:
        silhouette = -1
    dbscan_results[(conf['eps'], conf['min_samples'])] = silhouette

n_comps = len(dataCleaned['Top 10 (xTimes)'].unique())
gmm = GaussianMixture(n_components=n_comps, random_state=42)
gmm_labels = gmm.fit_predict(X_escalado)
gmm_silhouette = silhouette_score(X_escalado, gmm_labels)


print("Resultados DBSCAN:")
for conf, score in dbscan_results.items():
    print(f"Configuracion (eps={conf[0]}, min_samples={conf[1]}): Silhouette Score = {score:.2f}")

print(f"\nGMM Silhouette Score: {gmm_silhouette:.2f}")


Resultados DBSCAN:
Configuracion (eps=0.3, min_samples=5): Silhouette Score = 0.33
Configuracion (eps=0.5, min_samples=5): Silhouette Score = 0.55
Configuracion (eps=0.3, min_samples=10): Silhouette Score = 0.67
Configuracion (eps=0.5, min_samples=10): Silhouette Score = 0.76

GMM Silhouette Score: -0.25


Para poder realizar una comparación de los datos, en DBSCAN se utilizan distintos parametros, y se obtiene el Silhouette score, que se utiliza para evaluar la calidad de cada clustering. De esta manera podremos tener un parámetro para poder medir cada resultado y comparar ambos algoritmos. 

Como se puede observar, en DBSCAN va a variar en cada uno de los eps definidos, donde en al dar valores mas pequeños se va a obtener un Silhouette Score mas bajo, lo que indica que se formarán clusters mas pequeños y menos definidos (distintos entre sí). Al aumentar estos parametros de eps y min_samples, se obtendra un silhouette Score mas alto, lo que muestra que se formarán clusters mas grandes y definidos. Sin embargo, en GMM se muestra un Silhouette Score de de -0.25, que al dar un número negativo, va a mostrar que estos clusters están mal definidos.

Por lo tanto, GMM será mas eficiente para datos que se adapten más a distribuciones Gaussianas.